# Cleaning Folha Dataset

In [84]:
import pandas as pd
import numpy as np
import html
import re
from datasets import Dataset, DatasetDict, load_dataset

pd.set_option('display.max_rows', 120)

## 1. Load Dataset and Overview

In [2]:
dataset_filepath = "../data/folha_2015_2023.zip"
folha = pd.read_csv(dataset_filepath)
folha.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [3]:
# number of rows and columns
print(f"Rows: {folha.shape[0]}")
print(f"Columns: {folha.shape[1]}")

Rows: 532727
Columns: 6


## 2. Checking columns

### 2.1. title

In [4]:
# NaN titles
print(f"NaN title rows: {folha['title'].isna().sum()}")
print(f"% NaN title rows: {folha['title'].isna().sum() / len(folha) * 100} %")

NaN title rows: 0
% NaN title rows: 0.0 %


In [5]:
# check for double spaces, tabs or line breaks
cond = folha['title'].str.contains(r"  +|\t+|\n+")
folha.loc[cond, 'title']

43        \n\t\tDa escola ao 1º emprego: como buscar um ...
160       \n\t\tComo diferenciar ruídos que indicam prob...
163       \n\t\tAdvogado explica as regras sobre venda d...
165       \n\t\tMoradores da zona oeste são os mais sati...
171       \n\t\tMoradores recuperam praça no Jardim das ...
                                ...                        
125343    Grécia inicia o pagamento de  6,25 bilhões ao ...
126322    Comissão da UE propõe empréstimo de curto praz...
145679    Moradores esperam por obras públicas  na 'Ilha...
501558    Morcheeba retorna ao Brasil para  Nublu Jazz F...
504758    ​    Para leitores, Carnaval de São Paulo sobr...
Name: title, Length: 3931, dtype: object

In [6]:
# check for double spaces, tabs or line breaks
folha.loc[cond, 'title'].tail(1).values[0]

'\u200b    Para leitores, Carnaval de São Paulo sobrepõe dinheiro ao bem-estar da população'

### 2.2. text

In [9]:
# NaN text
print(f"NaN text rows: {folha['text'].isna().sum()}")
print(f"% NaN text rows: {folha['text'].isna().sum() / len(folha) * 100} %")

NaN text rows: 32009
% NaN text rows: 6.008518434395103 %


In [10]:
# check for double spaces, tabs or line breaks
cond = folha['text'].apply(lambda x: str(x)).str.contains(r"  +|\t+|\n+")
folha.loc[cond, 'text'].tail(1).values[0]

'Confira o que acontece no capítulo desta terça\xa0(2) em "Tempos de Amar", novela de Alcides Nogueira:Em Morros Verdes, Maria Vitória (Vitória Strada) insiste para que Inácio (Bruno Cabrerizo) mantenha seus planos de ir para o Brasil. José Augusto (Tony Ramos) elogia Inácio. Inácio se despede de Maria Vitória.No Rio, Conselheiro (Werner Shünemann) termina o romance com Celeste Hermínia (Marisa Orth).\xa0Passa-se mais um mês. Inácio chega ao Rio de Janeiro. Em Morros Verdes, Maria Vitória torce por notícias de Inácio.No Rio, Vicente\xa0(Bruno Ferrari) tenta convencer Conselheiro a procurar Celeste Hermínia. Teodoro pensa em cortejar a fadista. Inácio começa a trabalhar com Geraldo (Jackson Antunes).                         Em Morros Verdes, Maria Vitória sente uma forte tontura e é observada pela governanta. Delfina (Letícia Sabatella) afirma a Tereza (Olivia\xa0Torres) que ela receberá uma grande quantia para se casar. No Rio, Reinaldo ( Cassio Gabus Mendes)\xa0insiste para que Lucind

In [11]:
text = folha.loc[cond, 'text'].tail(10).values[0]
test = re.sub(r"\s+", " ", text)
test

'A saga da personagem de Carol Duarte é repleta de grandes acontecimentos: Ivana se descobre trans, encara o processo de readequação de gênero, muda seu nome para Ivan e fica grávido.Desde a revelação da gravidez, na última sexta (29), espectadores passaram a fazer algumas contas para identificar quando ocorreu a fecundação.Desde que Ivan (enquanto ainda era Ivana) transou com Cláudio (Gabriel Stalffer) muitas coisas aconteceram na novela. Internautas calcularam que o tempo transcorrido na trama seja de aproximadamente um ano. "Ivana grávida depois de séculos que o namorado dela foi embora. Gente, que espermatozóide forte esse", escreveu uma internauta no Twitter. Gloria Perez explicou que a contagem do tempo é diferente. Em entrevista ao portal "Extra", a autora disse que "em novela uma noite ou um dia podem durar uma semana inteira até". Depois daquela noite, Cláudio disse que ficaria um ano fora do país e agora o personagem já deve voltar para o folhetim. Então, de fato, quanto temp

### 2.3. date

In [12]:
# NaN data
print(f"NaN date rows: {folha['date'].isna().sum()}")
print(f"% NaN date rows: {folha['date'].isna().sum() / len(folha) * 100} %")

NaN date rows: 30117
% NaN date rows: 5.653364668958022 %


### 2.4. category

In [14]:
# NaN categories
print(f"NaN category rows: {folha['category'].isna().sum()}")
print(f"% NaN category rows: {folha['category'].isna().sum() / len(folha) * 100} ")

NaN category rows: 0
% NaN category rows: 0.0 


In [15]:
len(folha['category'].value_counts().sort_values(ascending = False))

112

In [16]:
# number of rows per category
folha['category'].value_counts().sort_values(ascending = False)

category
colunas                                            88528
mercado                                            77447
poder                                              51463
mundo                                              42016
cotidiano                                          39639
ilustrada                                          38162
esporte                                            35389
celebridades                                       17627
opiniao                                            14228
equilibrioesaude                                   11783
televisao                                           9774
internacional                                       7775
paineldoleitor                                      6575
sao-paulo                                           6181
educacao                                            5088
colunistas                                          5047
blogs                                               4926
saopaulo              

In [17]:
# colunas x colunistas (seems to be the same thing)
print(folha.query("category == 'colunas'").iloc[2, -1])
print(folha.query("category == 'colunistas'").iloc[0, -1])

http://www1.folha.uol.com.br/colunas/marceloleite/2017/10/1925096-cientista-da-psicodelico-para-minicerebros-e-eles-gostam.shtml
https://f5.folha.uol.com.br/colunistas/cristina-padiglione/2023/05/galvao-bueno-nao-tenho-por-que-pedir-desculpas-ao-neymar.shtml


In [18]:
# restaurantes x comidas (seems to be the same thing)
print(folha.query("category == 'restaurantes'").iloc[0, -1])
print(folha.query("category == 'comida'").iloc[1, -1])

https://guia.folha.uol.com.br/restaurantes/2023/05/restaurante-quincho-serve-menu-degustacao-apenas-com-sobremesas-em-sp.shtml
http://www1.folha.uol.com.br/comida/2017/04/1872819-eleven-madison-park-de-ny-e-eleito-o-melhor-restaurante-do-mundo.shtml


In [19]:
# cinemas-e-series x cinema x 42a-mostra-internacional-de-cinema x 41a-mostra-internacional-de-cinema (seems to be the same thing)
print(folha.query("category == 'cinema-e-series'").iloc[2, -1])
print(folha.query("category == 'cinema'").iloc[4, -1])
print(folha.query("category == '42a-mostra-internacional-de-cinema'").iloc[0, -1])
print(folha.query("category == '41a-mostra-internacional-de-cinema'").iloc[1, -1])

https://f5.folha.uol.com.br/cinema-e-series/2023/05/barbie-ganha-novo-trailer-com-mais-detalhes-da-historia-da-boneca-entrando-no-mundo-real.shtml
https://guia.folha.uol.com.br/cinema/2023/05/velozes-e-furiosos-10-domina-as-salas-de-cinema-em-sp-que-tem-5-estreias.shtml
https://guia.folha.uol.com.br/42a-mostra-internacional-de-cinema/2018/10/central-do-brasil-tem-sessao-com-fernanda-montenegro-na-mostra-confira-outras-dicas-do-dia.shtml
https://guia.folha.uol.com.br/41a-mostra-internacional-de-cinema/2017/11/veja-tres-filmes-em-destaque-no-ultimo-dia-da-mostra-de-cinema.shtml


In [20]:
# sao-paulo x saopaulo x o-melhor-de-sao-paulo (seems to be the same thing)
print(folha.query("category == 'sao-paulo'").iloc[9, -1])
print(folha.query("category == 'saopaulo'").iloc[30, -1])
print(folha.query("category == 'o-melhor-de-sao-paulo'").iloc[12, -1])

https://agora.folha.uol.com.br/sao-paulo/2021/11/arvore-de-natal-na-ponte-estaiada-em-sp-sera-inaugurada-neste-sabado-27.shtml
http://www1.folha.uol.com.br/saopaulo/2017/10/1922945-padrao-geometrico-conhecido-como-chevron-ajuda-a-dar-movimento-aos-ambientes.shtml
http://www1.folha.uol.com.br/o-melhor-de-sao-paulo/2017/casa-e-decoracao/2017/08/1910839-jovens-dedicam-se-a-ceramica-artesanal-e-produzem-pecas-cheias-de-estilo.shtml


## 3. Clean Dataset

In [21]:
folha_filtered = folha.copy()

In [22]:
def group_categories(df):
    """
    Group similar categories into one.
    """
    df.loc[df['category'] == 'equilibrioesaude', 'category'] = 'equilibrio'
    df.loc[df['category'] == 'tv', 'category'] = 'televisao'
    df.loc[df['category'] == 'mundo', 'category'] = 'internacional'
    df.loc[df['category'] == 'cinema', 'category'] = 'cinema-e-series'
    df.loc[df['category'] == 'equilibrio', 'category'] = 'equilibrio-e-saude'

    return df

In [23]:
def clean_text(df, column):
    """
    Cleans text column, removing tabs, line breaks, double spaces, unicode and html strings. Removes NaN values.
    """

    # remove NaN values
    df.dropna(subset = column, inplace = True)

    # remove tabs and line breaks
    df[column] = df[column].str.replace(r"\t+|\n+", "")

    # remove double spaces and strip
    df[column] = df[column].str.strip() \
                        .apply(lambda text: re.sub(r"\s+", " ", text))

    # remove html strings
    df[column] = df[column].apply(lambda text: html.unescape(text))

    return df

### 3.1. Filter categories

In [24]:
categories_to_drop = [
    "folhinha",
    "webstories",
    "seminariosfolha",
    "banco-de-dados",
    "sobretudo",
    "voceviu",
    "ilustrissima",
    "blogs",
    "colunistas",
    "sao-paulo",
    "saopaulo",
    "paineldoleitor",
    "opiniao",
    "ilustrada",
    "colunas"
]

In [25]:
aux = pd.DataFrame(folha['category'].value_counts().rename('qtd').sort_values(ascending=False))
aux = aux.query("qtd > 1000").reset_index()
aux = aux.query("~category.isin(@categories_to_drop)")
filtered_categories = aux['category'].unique()
filtered_categories

array(['mercado', 'poder', 'mundo', 'cotidiano', 'esporte',
       'celebridades', 'equilibrioesaude', 'televisao', 'internacional',
       'educacao', 'grana', 'tec', 'turismo', 'ambiente', 'ciencia',
       'cinema-e-series', 'musica', 'tv', 'empreendedorsocial',
       'podcasts', 'comida', 'cinema', 'equilibrio'], dtype=object)

In [26]:
print(f"Before filtering categories: {len(folha_filtered)}")
folha_filtered = folha_filtered.query("category.isin(@filtered_categories)").copy()
print(f"After filtering categories: {len(folha_filtered)}")

Before filtering categories: 532727
After filtering categories: 332930


### 3.2. Clean text

In [27]:
def clean_dataset(df):
    """
    Pipeline to clean dataset.
    """
    print(f"Rows in dataset: {len(df)}")
    # cleans title
    df = clean_text(df, 'title')
    print(f"Rows in dataset after title cleaning: {len(df)}")

    # cleans text
    df = clean_text(df, 'text')
    print(f"Rows in dataset after text cleaning: {len(df)}")

    # group categories
    df = group_categories(df)

    return df

In [28]:
# clean and export
folha_filtered = clean_dataset(folha_filtered)

Rows in dataset: 332930
Rows in dataset after title cleaning: 332930
Rows in dataset after text cleaning: 308862


### 3.3. Filter texts with less than X characters

In [56]:
print(folha_filtered.shape)
folha_filtered = folha_filtered.loc[folha_filtered['text'].str.len() > 300]
print(folha_filtered.shape)

(351969, 7)
(351969, 7)


### 3.4. Filter Columns

In [29]:
folha_filtered = folha_filtered.loc[:, ["title", "text", "date", "category", "link"]]
folha_filtered.head()

,title,text,date,category,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,http://www1.folha.uol.com.br/mercado/2017/10/1...
5,"Em encontro, Bono pergunta a Macri sobre argen...","O vocalista da banda irlandesa U2, Bono, fez u...",2017-09-10,internacional,http://www1.folha.uol.com.br/mundo/2017/10/192...
6,"Posso sair do Brasil quando e como quiser, diz...",O italiano Cesare Battisti disse nesta segunda...,2017-09-10,poder,http://www1.folha.uol.com.br/poder/2017/10/192...


### 3.5. Check changes

In [104]:
def calculate_metrics(df):
    # calculate descriptive statistics
    metrics = df.groupby('category')['text'] \
                .apply(lambda x: x.str.len().describe()) \
                .reset_index()
    metrics = pd.pivot_table(
        metrics,
        values = "text",
        index = "category",
        columns = "level_1"
        ) \
        .add_suffix("_len_text") \
        .reset_index()
    
    # calculate percentage
    df.rename(columns = {'count_len_text' : 'count'}, inplace = True)
    pct = df['category'].value_counts(normalize = True)

    # calculate mean title length
    mean_len_title = df.groupby('category')['title'] \
                       .apply(lambda x: np.mean(x.str.len())) \
                       .rename("mean_len_title") \
                       .round() \
                       .astype(int)

    # join metrics tables
    pct_mean_len_title = pct.join(mean_len_title)
    metrics = metrics.merge(
        pct_mean_len_title,
        left_on = "category",
        right_index = True
        )
    metrics['count'] = metrics['count'].astype(int)
    metrics = metrics[
        [
            "category", "count", "mean_len_title", "25%_len_text", 
            "50%_len_text", "75%_len_text", "max_len_text", 
            "mean_len_text", "min_len_text", "std_len_text", 
            "proportion"
        ]
    ].sort_values(by = "count", ascending = False)

    return metrics

In [105]:
metrics = calculate_metrics(folha_filtered)
metrics

,category,count,mean_len_title,25%_len_text,50%_len_text,75%_len_text,max_len_text,mean_len_text,min_len_text,std_len_text,proportion
12,mercado,62139,69,2059.00,3131.0,4570.00,102232.0,3613.667182,301.0,2345.886226,0.176547
11,internacional,51617,71,1686.00,2871.0,4491.00,50032.0,3440.521979,302.0,2412.723583,0.146652
15,poder,51289,72,2280.00,3411.0,5201.00,215128.0,4042.673361,301.0,2996.213376,0.145720
5,cotidiano,40392,70,1883.00,2869.0,4318.00,37019.0,3323.078209,301.0,2012.574652,0.114760
9,esporte,35327,68,1448.00,2280.0,3419.50,57574.0,2721.891698,301.0,1885.982947,0.100370
1,celebridades,32274,74,901.00,1345.0,1870.00,20678.0,1582.844054,301.0,1203.821390,0.091696
17,televisao,25841,73,735.00,1249.0,2052.00,34127.0,1637.577764,301.0,1417.285068,0.073418
8,equilibrio-e-saude,13690,74,2764.00,4213.0,5920.00,31998.0,4691.704456,305.0,2800.802617,0.038895
3,cinema-e-series,6100,76,1291.00,1988.5,3587.75,52737.0,3110.174754,328.0,3512.528648,0.017331
6,educacao,5150,70,2468.25,3633.5,5029.75,33713.0,3940.905243,302.0,2083.612807,0.014632


## 4. Export and push to hub

### 4.1. Export to CSV (.zip)

In [ ]:
# Export file
# filename = 'folha_2015_2023_clean_cats'
# print(f"Exporting file {filename}")
# compression_options = dict(method='zip', archive_name=f'{filename}.csv')
# folha_filtered.to_csv(f'../data/{filename}.zip', compression=compression_options, index = False)

### 4.2. Export to Hub

In [17]:
dataset_filepath = "../data/folha_2013_2023_clean_cats.zip"
folha_filtered = pd.read_csv(dataset_filepath)
print(folha_filtered.columns)
print(folha_filtered.shape)

Index(['title', 'text', 'date', 'category', 'subcategory', 'link'], dtype='object')
(354534, 6)


In [37]:
rename_dict = {
    'mercado':'Mercado',
    'poder': 'Poder',
    'internacional': 'Internacional',
    'cotidiano': 'Cotidiano',
    'esporte': 'Esporte',
    'celebridades': 'Celebridades',
    'televisao': 'Televisão',
    'equilibrio-e-saude': 'Equilibrio e Saúde',
    'cinema-e-series': 'Cinema e Séries',
    'educacao': 'Educação',
    'grana': 'Grana ou Dinheiro',
    'ambiente': 'Meio Ambiente',
    'tec': 'Tecnologia',
    'ciencia': 'Ciência',
    'turismo': 'Turismo',
    'musica': 'Música',
    'empreendedorsocial': 'Empreendedorismo Social',
    'podcasts': 'Podcasts',
    'comida': 'Comida'
}

folha_filtered['category_natural_language'] = folha_filtered['category'].map(rename_dict)
folha_filtered['category_natural_language'].unique()

array(['Equilibrio e Saúde', 'Cotidiano', 'Mercado', 'Meio Ambiente',
       'Comida', 'Internacional', 'Poder', 'Celebridades', 'Televisão',
       'Podcasts', 'Educação', 'Esporte', 'Música', 'Ciência',
       'Cinema e Séries', 'Turismo', 'Tecnologia',
       'Empreendedorismo Social', 'Grana ou Dinheiro'], dtype=object)

In [108]:
test = folha_filtered.sample(frac = 0.5, random_state = 42)
metrics_test = calculate_metrics(test)
metrics_test

,category,count,mean_len_title,25%_len_text,50%_len_text,75%_len_text,max_len_text,mean_len_text,min_len_text,std_len_text,proportion
12,mercado,31033,69,2069.00,3142.0,4601.00,102232.0,3637.836980,302.0,2431.093609,0.176340
11,internacional,25883,71,1687.00,2871.0,4492.50,36758.0,3443.931074,302.0,2411.889000,0.147076
15,poder,25638,72,2273.25,3409.0,5166.00,140692.0,4030.479601,301.0,2905.277986,0.145684
5,cotidiano,20252,70,1889.00,2859.0,4298.00,33837.0,3320.685019,301.0,2039.457957,0.115079
9,esporte,17560,68,1446.00,2276.5,3409.00,26245.0,2710.484226,304.0,1841.851998,0.099782
1,celebridades,16189,74,900.00,1344.0,1873.00,20678.0,1577.593489,301.0,1177.447976,0.091991
17,televisao,12840,73,727.00,1240.0,2052.00,22757.0,1636.208723,301.0,1420.810903,0.072961
8,equilibrio-e-saude,6856,74,2765.00,4193.5,5920.00,28662.0,4698.372229,305.0,2838.035506,0.038958
3,cinema-e-series,3094,75,1291.00,1987.0,3694.75,52737.0,3118.491920,385.0,3339.381548,0.017581
6,educacao,2601,70,2508.00,3693.0,5043.00,22681.0,3955.819300,365.0,2012.898619,0.014780


In [107]:
train = folha_filtered.loc[~folha_filtered.index.isin(test.index)]
metrics_train = calculate_metrics(train)
metrics_train

,category,count,mean_len_title,25%_len_text,50%_len_text,75%_len_text,max_len_text,mean_len_text,min_len_text,std_len_text,proportion
12,mercado,30854,69,2051.00,3133.0,4561.00,102232.0,3612.412264,301.0,2368.103612,0.175322
11,internacional,25907,71,1684.00,2876.0,4472.00,50032.0,3434.469448,302.0,2407.892863,0.147211
15,poder,25680,72,2290.00,3419.0,5215.00,136497.0,4044.994470,301.0,2821.195553,0.145922
5,cotidiano,20192,70,1875.00,2865.0,4318.25,37019.0,3322.238758,301.0,2021.196188,0.114737
9,esporte,17744,68,1456.75,2288.0,3419.50,44172.0,2728.565487,303.0,1863.002703,0.100827
1,celebridades,16088,74,897.75,1343.0,1869.00,20678.0,1584.238687,301.0,1219.144578,0.091417
17,televisao,13017,73,734.00,1256.0,2048.00,21610.0,1633.716601,301.0,1384.572165,0.073967
8,equilibrio-e-saude,6815,74,2786.50,4225.0,5946.00,29320.0,4717.689949,331.0,2814.713164,0.038725
3,cinema-e-series,3034,76,1269.25,1939.0,3480.00,52575.0,3007.184245,328.0,3442.398635,0.017240
6,educacao,2607,70,2439.00,3611.0,5014.00,22857.0,3912.515535,302.0,2035.084983,0.014814


In [87]:
train_dataset = Dataset.from_pandas(train[["title", "text", "date", "category", "category_natural_language", "link"]])
test_dataset = Dataset.from_pandas(test[["title", "text", "date", "category", "category_natural_language", "link"]])
train_dataset = train_dataset.remove_columns("__index_level_0__")
test_dataset = test_dataset.remove_columns("__index_level_0__")

In [88]:
folha_dataset = DatasetDict(
    {
        'train': train_dataset,
        'test': test_dataset 
    }
)

In [89]:
print(folha_dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'date', 'category', 'category_natural_language', 'link'],
        num_rows: 175985
    })
    test: Dataset({
        features: ['title', 'text', 'date', 'category', 'category_natural_language', 'link'],
        num_rows: 175984
    })
})


In [95]:
# folha_dataset.push_to_hub("iara-project/news-articles-ptbr-dataset")

Pushing dataset shards to the dataset hub: 100%|██████████| 2/2 [01:25<00:00, 42.83s/it]
